Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/forecasting-backtest-many-models/auto-ml-forecasting-backtest-many-models.png)

# Many Models with Backtesting - Automated ML
**_Backtest many models time series forecasts with Automated Machine Learning_**

---

For this notebook we are using a synthetic dataset to demonstrate the back testing in many model scenario. This allows us to check historical performance of AutoML on a historical data. To do that we step back on the backtesting period by the data set several times and split the data to train and test sets. Then these data sets are used for training and evaluation of model.<br>

Thus, it is a quick way of evaluating AutoML as if it was in production. Here, we do not test historical performance of a particular model, for this see the [notebook](../forecasting-backtest-single-model/auto-ml-forecasting-backtest-single-model.ipynb). Instead, the best model for every backtest iteration can be different since AutoML chooses the best model for a given training set.

![Backtesting](Backtesting.png)

**NOTE: There are limits on how many runs we can do in parallel per workspace, and we currently recommend to set the parallelism to maximum of 320 runs per experiment per workspace. If users want to have more parallelism and increase this limit they might encounter Too Many Requests errors (HTTP 429).**

### Prerequisites
You'll need to create a compute Instance by following [these](https://learn.microsoft.com/en-us/azure/machine-learning/v1/how-to-create-manage-compute-instance?tabs=python) instructions.

## 1.0 Set up workspace, datastore, experiment

In [ ]:
import os

import azureml.core
from azureml.core import Workspace, Datastore
import numpy as np
import pandas as pd

from pandas.tseries.frequencies import to_offset

# Set up your workspace
ws = Workspace.from_config()
ws.get_details()

# Set up your datastores
dstore = ws.get_default_datastore()

output = {}
output["SDK version"] = azureml.core.VERSION
output["Subscription ID"] = ws.subscription_id
output["Workspace"] = ws.name
output["Resource Group"] = ws.resource_group
output["Location"] = ws.location
output["Default datastore name"] = dstore.name
output["SDK Version"] = azureml.core.VERSION
pd.set_option("display.max_colwidth", None)
outputDf = pd.DataFrame(data=output, index=[""])
outputDf.T

This notebook is compatible with Azure ML SDK version 1.35.1 or later.

In [ ]:
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

### Choose an experiment

In [ ]:
from azureml.core import Experiment

experiment = Experiment(ws, "automl-many-models-backtest")

print("Experiment name: " + experiment.name)

## 2.0 Data

#### 2.1 Data generation
For this notebook we will generate the artificial data set with two [time series IDs](https://docs.microsoft.com/en-us/python/api/azureml-automl-core/azureml.automl.core.forecasting_parameters.forecastingparameters?view=azure-ml-py). Then we will generate backtest folds and will upload it to the default BLOB storage and create a [TabularDataset](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabular_dataset.tabulardataset?view=azure-ml-py)

In [ ]:
# simulate data: 2 grains - 700
TIME_COLNAME = "date"
TARGET_COLNAME = "value"
TIME_SERIES_ID_COLNAME = "ts_id"

sample_size = 700
# Set the random seed for reproducibility of results.
np.random.seed(20)
X1 = pd.DataFrame(
    {
        TIME_COLNAME: pd.date_range(start="2018-01-01", periods=sample_size),
        TARGET_COLNAME: np.random.normal(loc=100, scale=20, size=sample_size),
        TIME_SERIES_ID_COLNAME: "ts_A",
    }
)
X2 = pd.DataFrame(
    {
        TIME_COLNAME: pd.date_range(start="2018-01-01", periods=sample_size),
        TARGET_COLNAME: np.random.normal(loc=100, scale=20, size=sample_size),
        TIME_SERIES_ID_COLNAME: "ts_B",
    }
)

X = pd.concat([X1, X2], ignore_index=True, sort=False)
print("Simulated dataset contains {} rows \n".format(X.shape[0]))
X.head()

Now we will generate 8 backtesting folds with backtesting period of 7 days and with the same forecasting horizon. We will add the column "backtest_iteration", which will identify the backtesting period by the last training date.

In [ ]:
offset_type = "7D"
NUMBER_OF_BACKTESTS = 8  # number of train/test sets to generate

dfs_train = []
dfs_test = []
for ts_id, df_one in X.groupby(TIME_SERIES_ID_COLNAME):

    data_end = df_one[TIME_COLNAME].max()

    for i in range(NUMBER_OF_BACKTESTS):
        train_cutoff_date = data_end - to_offset(offset_type)
        df_one = df_one.copy()
        df_one["backtest_iteration"] = "iteration_" + str(train_cutoff_date)
        train = df_one[df_one[TIME_COLNAME] <= train_cutoff_date]
        test = df_one[
            (df_one[TIME_COLNAME] > train_cutoff_date)
            & (df_one[TIME_COLNAME] <= data_end)
        ]
        data_end = train[TIME_COLNAME].max()
        dfs_train.append(train)
        dfs_test.append(test)

X_train = pd.concat(dfs_train, sort=False, ignore_index=True)
X_test = pd.concat(dfs_test, sort=False, ignore_index=True)

#### 2.2 Create the Tabular Data Set.

A Datastore is a place where data can be stored that is then made accessible to a compute either by means of mounting or copying the data to the compute target.

Please refer to [Datastore](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.datastore(class)?view=azure-ml-py) documentation on how to access data from Datastore.

In this next step, we will upload the data and create a TabularDataset.

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

ds = ws.get_default_datastore()
# Upload saved data to the default data store.
train_data = TabularDatasetFactory.register_pandas_dataframe(
    X_train, target=(ds, "data_mm"), name="data_train"
)
test_data = TabularDatasetFactory.register_pandas_dataframe(
    X_test, target=(ds, "data_mm"), name="data_test"
)

## 3.0 Build the training pipeline
Now that the dataset, WorkSpace, and datastore are set up, we can put together a pipeline for training.

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

### Choose a compute target

You will need to create a [compute target](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-set-up-training-targets#amlcompute) for your AutoML run. In this tutorial, you create AmlCompute as your training compute resource.

\*\*Creation of AmlCompute takes approximately 5 minutes.**

If the AmlCompute with that name is already in your workspace this code will skip the creation process. As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read this [article](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-manage-quotas) on the default limits and how to request more quota.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# Name your cluster
compute_name = "backtest-mm"


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("Found compute target: " + compute_name)
else:
    print("Creating a new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_DS12_V2", max_nodes=6
    )
    # Create the compute target
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    # Can poll for a minimum number of nodes and for a specific timeout.
    # If no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=20
    )

    # For a more detailed view of current cluster status, use the 'status' property
    print(compute_target.status.serialize())

### Set up training parameters

We need to provide ``ForecastingParameters``, ``AutoMLConfig`` and ``ManyModelsTrainParameters`` objects. For the forecasting task we also need to define several settings including the name of the time column, the maximum forecast horizon, and the partition column name(s) definition.

#### ``ForecastingParameters`` arguments
| Property                           | Description|
| :---------------                   | :------------------- |
| **forecast_horizon**               | The forecast horizon is how many periods forward you would like to forecast. This integer horizon is in units of the timeseries frequency (e.g. daily, weekly). Periods are inferred from your data. |
| **time_column_name**               | The name of your time column. |
| **time_series_id_column_names**    | The column names used to uniquely identify timeseries in data that has multiple rows with the same timestamp. |
| **cv_step_size**                   | Number of periods between two consecutive cross-validation folds. The default value is \"auto\", in which case AutoMl determines the cross-validation step size automatically, if a validation set is not provided. Or users could specify an integer value. |

#### ``AutoMLConfig`` arguments
| Property                           | Description|
| :---------------                   | :------------------- |
| **task**                           | forecasting |
| **primary_metric**                 | This is the metric that you want to optimize.<br> Forecasting supports the following primary metrics <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i> |
| **blocked_models**                 | Blocked models won't be used by AutoML. |
| **iteration_timeout_minutes**      | Maximum amount of time in minutes that the model can train. This is optional but provides customers with greater control on exit criteria. |
| **iterations**                     | Number of models to train. This is optional but provides customers with greater control on exit criteria. |
| **experiment_timeout_hours**       | Maximum amount of time in hours that each experiment can take before it terminates. This is optional but provides customers with greater control on exit criteria. **It does not control the overall timeout for the pipeline run, instead controls the timeout for each training run per partitioned time series.** |
| **label_column_name**              | The name of the label column. |
| **n_cross_validations**            | Number of cross validation splits. The default value is \"auto\", in which case AutoMl determines the number of cross-validations automatically, if a validation set is not provided. Or users could specify an integer value. Rolling Origin Validation is used to split time-series in a temporally consistent way. |
| **enable_early_stopping**          | Flag to enable early termination if the primary metric is no longer improving. |
| **enable_engineered_explanations** | Engineered feature explanations will be downloaded if enable_engineered_explanations flag is set to True. By default it is set to False to save storage space. |
| **track_child_runs**               | Flag to disable tracking of child runs. Only best run is tracked if the flag is set to False (this includes the model and metrics of the run). |
| **pipeline_fetch_max_batch_size**  | Determines how many pipelines (training algorithms) to fetch at a time for training, this helps reduce throttling when training at large scale. |


#### ``ManyModelsTrainParameters`` arguments
| Property                           | Description|
| :---------------                   | :------------------- |
| **automl_settings**                | The ``AutoMLConfig`` object defined above. |
| **partition_column_names**         | The names of columns used to group your models. For timeseries, the groups must not split up individual time-series. That is, each group must contain one or more whole time-series. |

In [ ]:
from azureml.train.automl.runtime._many_models.many_models_parameters import (
    ManyModelsTrainParameters,
)
from azureml.automl.core.forecasting_parameters import ForecastingParameters
from azureml.train.automl.automlconfig import AutoMLConfig

partition_column_names = [TIME_SERIES_ID_COLNAME, "backtest_iteration"]

forecasting_parameters = ForecastingParameters(
    time_column_name=TIME_COLNAME,
    forecast_horizon=6,
    time_series_id_column_names=partition_column_names,
    cv_step_size="auto",
)

automl_settings = AutoMLConfig(
    task="forecasting",
    primary_metric="normalized_root_mean_squared_error",
    iteration_timeout_minutes=10,
    iterations=15,
    experiment_timeout_hours=0.25,
    label_column_name=TARGET_COLNAME,
    n_cross_validations="auto",  # Feel free to set to a small integer (>=2) if runtime is an issue.
    track_child_runs=False,
    forecasting_parameters=forecasting_parameters,
)


mm_paramters = ManyModelsTrainParameters(
    automl_settings=automl_settings, partition_column_names=partition_column_names
)

### Set up many models pipeline

Parallel run step is leveraged to train multiple models at once. To configure the ParallelRunConfig you will need to determine the appropriate number of workers and nodes for your use case. The process_count_per_node is based off the number of cores of the compute VM. The node_count will determine the number of master nodes to use, increasing the node count will speed up the training process.

| Property                           | Description|
| :---------------                   | :------------------- |
| **experiment**                     | The experiment used for training. |
| **train_data**                     | The file dataset to be used as input to the training run. |
| **node_count**                     | The number of compute nodes to be used for running the user script. We recommend to start with 3 and increase the node_count if the training time is taking too long. |
| **process_count_per_node**         | Process count per node, we recommend 2:1 ratio for number of cores: number of processes per node. eg. If node has 16 cores then configure 8 or less process count per node or optimal performance. |
| **train_pipeline_parameters**      | The set of configuration parameters defined in the previous section. |
| **run_invocation_timeout**         | Maximum amount of time in seconds that the ``ParallelRunStep`` class is allowed. This is optional but provides customers with greater control on exit criteria. This must be greater than ``experiment_timeout_hours`` by at least 300 seconds. |

Calling this method will create a new aggregated dataset which is generated dynamically on pipeline execution.

**Note**: Total time taken for the **training step** in the pipeline to complete = $ \frac{t}{ p \times n } \times ts $
where,
- $ t $ is time taken for training one partition (can be viewed in the training logs)
- $ p $ is ``process_count_per_node``
- $ n $ is ``node_count``
- $ ts $ is total number of partitions in time series based on ``partition_column_names``

In [ ]:
from azureml.contrib.automl.pipeline.steps import AutoMLPipelineBuilder


training_pipeline_steps = AutoMLPipelineBuilder.get_many_models_train_steps(
    experiment=experiment,
    train_data=train_data,
    compute_target=compute_target,
    node_count=2,
    process_count_per_node=2,
    run_invocation_timeout=1200,
    train_pipeline_parameters=mm_paramters,
)

In [ ]:
from azureml.pipeline.core import Pipeline

training_pipeline = Pipeline(ws, steps=training_pipeline_steps)

### Submit the pipeline to run
Next we submit our pipeline to run. The whole training pipeline takes about 20 minutes using a STANDARD_DS12_V2 VM with our current ParallelRunConfig setting.

In [ ]:
training_run = experiment.submit(training_pipeline)

In [ ]:
training_run.wait_for_completion(show_output=False)

Check the run status, if training_run is in completed state, continue to next section. Otherwise, check the portal for failures.

## 4.0 Backtesting
Now that we selected the best AutoML model for each backtest fold, we will use these models to generate the forecasts and compare with the actuals.

### Set up output dataset for inference data
Output of inference can be represented as [OutputFileDatasetConfig](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.output_dataset_config.outputdatasetconfig?view=azure-ml-py) object and OutputFileDatasetConfig can be registered as a dataset. 

In [ ]:
from azureml.data import OutputFileDatasetConfig

output_inference_data_ds = OutputFileDatasetConfig(
    name="many_models_inference_output",
    destination=(dstore, "backtesting/inference_data/"),
).register_on_complete(name="backtesting_data_ds")

For many models we need to provide the ManyModelsInferenceParameters object.

#### ``ManyModelsInferenceParameters`` arguments
| Property                           | Description|
| :---------------                   | :------------------- |
| **partition_column_names**         | List of column names that identifies groups. |
| **target_column_name**             | \[Optional] Column name only if the inference dataset has the target. |
| **time_column_name**               | \[Optional] Time column name only if it is timeseries. |
| **inference_type**                 | \[Optional] Which inference method to use on the model. Possible values are 'forecast', 'predict_proba', and 'predict'. |
| **forecast_mode**                  | \[Optional] The type of forecast to be used, either 'rolling' or 'recursive'; defaults to 'recursive'. |
| **step**                           | \[Optional] Number of periods to advance the forecasting window in each iteration **(for rolling forecast only)**; defaults to 1. |

#### ``get_many_models_batch_inference_steps`` arguments
| Property                           | Description|
| :---------------                   | :------------------- |
| **experiment**                     | The experiment used for inference run. |
| **inference_data**                 | The data to use for inferencing. It should be the same schema as used for training.
| **compute_target**                 | The compute target that runs the inference pipeline. |
| **node_count**                     | The number of compute nodes to be used for running the user script. We recommend to start with the number of cores per node (varies by compute sku). |
| **process_count_per_node**         | \[Optional] The number of processes per node. By default it's 2 (should be at most half of the number of cores in a single node of the compute cluster that will be used for the experiment).
| **inference_pipeline_parameters**  | \[Optional] The ``ManyModelsInferenceParameters`` object defined above. |
| **append_row_file_name**           | \[Optional] The name of the output file (optional, default value is 'parallel_run_step.txt'). Supports 'txt' and 'csv' file extension. A 'txt' file extension generates the output in 'txt' format with space as separator without column names. A 'csv' file extension generates the output in 'csv' format with comma as separator and with column names. |
| **train_run_id**                   | \[Optional] The run id of the **training pipeline**. By default it is the latest successful training pipeline run in the experiment. |
| **train_experiment_name**          | \[Optional] The train experiment that contains the train pipeline. This one is only needed when the train pipeline is not in the same experiement as the inference pipeline. |
| **run_invocation_timeout**         | \[Optional] Maximum amount of time in seconds that the ``ParallelRunStep`` class is allowed. This is optional but provides customers with greater control on exit criteria. |
| **output_datastore**               | \[Optional] The ``Datastore`` or ``OutputDatasetConfig`` to be used for output. If specified any pipeline output will be written to that location. If unspecified the default datastore will be used. |
| **arguments**                      | \[Optional] Arguments to be passed to inference script. Possible argument is '--forecast_quantiles' followed by quantile values. |

In [ ]:
from azureml.contrib.automl.pipeline.steps import AutoMLPipelineBuilder
from azureml.train.automl.runtime._many_models.many_models_parameters import (
    ManyModelsInferenceParameters,
)

mm_parameters = ManyModelsInferenceParameters(
    partition_column_names=partition_column_names,
    time_column_name=TIME_COLNAME,
    target_column_name=TARGET_COLNAME,
)

output_file_name = "parallel_run_step.csv"

inference_steps = AutoMLPipelineBuilder.get_many_models_batch_inference_steps(
    experiment=experiment,
    inference_data=test_data,
    node_count=2,
    process_count_per_node=2,
    compute_target=compute_target,
    run_invocation_timeout=300,
    output_datastore=output_inference_data_ds,
    train_run_id=training_run.id,
    train_experiment_name=training_run.experiment.name,
    inference_pipeline_parameters=mm_parameters,
    append_row_file_name=output_file_name,
)

In [ ]:
from azureml.pipeline.core import Pipeline

inference_pipeline = Pipeline(ws, steps=inference_steps)

In [ ]:
inference_run = experiment.submit(inference_pipeline)
inference_run.wait_for_completion(show_output=False)

## 5.0 Retrieve results and calculate metrics

The pipeline returns one file with the predictions for each times series ID and outputs the result to the forecasting_output Blob container. The details of the blob container is listed in 'forecasting_output.txt' under Outputs+logs. 

The next code snippet does the following:
1. Downloads the contents of the output folder that is passed in the parallel run step 
2. Reads the parallel_run_step.txt file that has the predictions as pandas dataframe 
3. Saves the table in csv format and 
4. Displays the top 10 rows of the predictions

In [ ]:
from azureml.contrib.automl.pipeline.steps.utilities import get_output_from_mm_pipeline

PREDICTION_COLNAME = "Predictions"
forecasting_results_name = "forecasting_results"
forecasting_output_name = "many_models_inference_output"
forecast_file = get_output_from_mm_pipeline(
    inference_run, forecasting_results_name, forecasting_output_name, output_file_name
)
df = pd.read_csv(forecast_file, parse_dates=[0])
print(
    "Prediction has ", df.shape[0], " rows. Here the first 10 rows are being displayed."
)
# Save the csv file to read it in the next step.
df.rename(
    columns={TARGET_COLNAME: "actual_level", PREDICTION_COLNAME: "predicted_level"},
    inplace=True,
)
df.to_csv(os.path.join(forecasting_results_name, "forecast.csv"), index=False)
df.head(10)

## View metrics
We will read in the obtained results and run the helper script, which will generate metrics and create the plots of predicted versus actual values.

In [ ]:
from assets.score import calculate_scores_and_build_plots

backtesting_results = "backtesting_mm_results"
os.makedirs(backtesting_results, exist_ok=True)
calculate_scores_and_build_plots(
    forecasting_results_name,
    backtesting_results,
    automl_settings.as_serializable_dict(),
)
pd.DataFrame({"File": os.listdir(backtesting_results)})

The directory contains a set of files with results:
- forecast.csv contains forecasts for all backtest iterations. The backtest_iteration column contains iteration identifier with the last training date as a suffix
- scores.csv contains all metrics. If data set contains several time series, the metrics are given for all combinations of time series id and iterations, as well as scores for all iterations and time series ids, which are marked as "all_sets"
- plots_fcst_vs_actual.pdf contains the predictions vs forecast plots for each iteration and, eash time series is saved as separate plot.

For demonstration purposes we will display the table of metrics for one of the time series with ID "ts0". We will create the utility function, which will build the table with metrics.

In [ ]:
def get_metrics_for_ts(all_metrics, ts):
    """
    Get the metrics for the time series with ID ts and return it as pandas data frame.

    :param all_metrics: The table with all the metrics.
    :param ts: The ID of a time series of interest.
    :return: The pandas DataFrame with metrics for one time series.
    """
    results_df = None
    for ts_id, one_series in all_metrics.groupby("time_series_id"):
        if not ts_id.startswith(ts):
            continue
        iteration = ts_id.split("|")[-1]
        df = one_series[["metric_name", "metric"]]
        df.rename({"metric": iteration}, axis=1, inplace=True)
        df.set_index("metric_name", inplace=True)
        if results_df is None:
            results_df = df
        else:
            results_df = results_df.merge(
                df, how="inner", left_index=True, right_index=True
            )
    results_df.sort_index(axis=1, inplace=True)
    return results_df


metrics_df = pd.read_csv(os.path.join(backtesting_results, "scores.csv"))
ts = "ts_A"
get_metrics_for_ts(metrics_df, ts)

Forecast vs actuals plots.

In [ ]:
from IPython.display import IFrame

IFrame("./backtesting_mm_results/plots_fcst_vs_actual.pdf", width=800, height=300)